## Upsell Optimization: Bridging BI Insights and Customer Eligibility with AI

#### Contents

- [Step 1: Import Necessary Libraries](#Step-1-Import-necessary-libraries)
- [Step 2: Load the data files](#Step-2-Load-the-data-files)
- [Step 3: Get a glimpse of data](#Step-3-Get-a-glimpse-of-data)
- [Step 4:  Function to implement logics](#Step-4-Function-to-implement-logics)
- [Step 5:  Join the dataframes to give complete solution](#Step-5-Join-the-dataframes-to-give-complete-solution)


#### Step 1: Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np

#### Step 2: Load the data files

In [2]:
customer_data = pd.read_csv('Business Translation Case_Customer Data.csv', delimiter=';')
restrict_data = pd.read_csv('Business Translation Case_Tec Restrictions.csv', delimiter=';')
action_data = pd.read_csv('Business Translation Case_Action Space.csv', delimiter=';')

#### Step 3: Get a glimpse of data

In [3]:
customer_data.iloc[:,:9]

,CUSTOMER_ID,PRODUCT_RANGE,PRODUCT,CONTRACT_START_DT,SPEED_MB,TECH,BEST_TECH,ROUTER,ROUTER_TYPE
0,329 08 509,2015,Single Flat,10.11.2019,100,Cable,Cable,Arris TG2492S,basic
1,235 09 840,2018,Double Flat,08.03.2018,50,FTTC,FTTC,AVM FRITZ!Box 7430,basic
2,192 85 309,2015,Tripple Flat,21.07.2019,25,BSA,BSA,unknown,unknown
3,420 95 309,2012,Single Flat,22.09.2019,50,FTTH,FTTH,AVM FRITZ!Box 7390,basic
4,309 50 932,2018,Double Flat,15.01.2020,18,FTTC,FTTH,AVM FRITZ!Box 7590,premium
5,239 85 092,2018,Single Flat,18.12.2018,25,BSA,FTTH,AVM FRITZ!Box 7583,premium
6,230 95 081,2012,Double Flat,05.05.2019,100,FTTH,FTTH,AVM FRITZ!Box 7390,basic
7,595 80 238,2015,Double Flat,21.11.2019,100,FTTH,FTTH,unknown,unknown
8,239 50 098,2018,Double Flat,07.06.2019,50,FTTH,FTTH,unknown,unknown
9,915 80 908,2018,Single Flat,23.08.2019,25,BSA,G.fast,AVM FRITZ!Box 7530,basic


In [ ]:
restrict_data

,KOMBI_ID,TECHNOLOGIE_GRUPPE,BANDBREITE_MB,ENDGERAET
0,1,BSA,50,AVM FRITZ!Box 7430
1,2,BSA,100,AVM FRITZ!Box 7430
2,3,BSA,50,AVM FRITZ!Box 7490
3,4,BSA,100,AVM FRITZ!Box 7490
4,5,BSA,50,AVM FRITZ!Box 7530
...,...,...,...,...
93,94,G.fast,50,AVM FRITZ!Box 7583
94,95,G.fast,100,AVM FRITZ!Box 7583
95,96,G.fast,250,AVM FRITZ!Box 7583
96,97,G.fast,500,AVM FRITZ!Box 7583


In [5]:
action_data.head(10)

,ACTION_ID,PROMOTION_TELEFONFLAT,SPEED_MB_TARGET,TECHNOLOGY_SWITCH,ROUTER_SWITCH,ROUTER_TYPE
0,A_01,0,50,0,0,no change
1,A_02,0,50,0,1,basic
2,A_03,0,50,0,1,premium
3,A_04,1,50,0,0,no change
4,A_05,1,50,0,1,basic
5,A_06,1,50,0,1,premium
6,A_07,0,100,0,0,no change
7,A_08,0,100,0,1,basic
8,A_09,0,100,0,1,premium
9,A_10,1,100,0,0,no change


#### Step 4:  Function to implement logics



In [6]:
# For simplicity sake each action is treated seperately and solved
def get_promotion(customer_row, action_data):
    
    '''
    This function returns the posible 'PROMOTION_TELEFONFLAT' actions
    
    params:
    Input: 
    
    customer_row: takes a single row from customer_data data frame of the column 'PRODUCT'
    action_data: action dataframe
    
    Output:
    
    returns all possible actions(list) for promotion using the telefon flat
    '''
    
    if customer_row.strip()=='Single Flat':
        return action_data[action_data['PROMOTION_TELEFONFLAT']==0]['ACTION_ID'].tolist()
    else:
        return action_data[action_data['PROMOTION_TELEFONFLAT']==1]['ACTION_ID'].tolist()

def get_speed(customer_row, restrict_data, action_data):
    
    '''
    This function returns the posible 'speed promotion' actions
    
    params:
    Input: 
    
    customer_row: takes a single row from customer_data data frame of the column 'SPEED_MB' and 'TECH'
    restrict_data: tech restrction dataframe
    action_data: action dataframe
    
    Output:
    
    returns all possible actions(list) for promotion using the speed
    '''
    
    current_speed, current_tech = customer_row
    x = restrict_data[(restrict_data['TECHNOLOGIE_GRUPPE'] == current_tech) & (restrict_data['BANDBREITE_MB']>current_speed)]
    possible_speeds = x['BANDBREITE_MB'].unique()
    
    return action_data[action_data['SPEED_MB_TARGET'].isin(possible_speeds)]['ACTION_ID'].tolist()
    
def get_technology(customer_row, action_data):
    
    '''
    This function returns the posible 'technology promotion' actions
    
    params:
    Input: 
    
    customer_row: takes a single row from customer_data data frame of the column 'TECH' and 'BEST_TECH'
    action_data: action dataframe
    
    Output:
    
    returns all possible actions(list) for promotion using the technology
    '''
    
    current_tech, best_tech = customer_row
    if current_tech == best_tech:
        return action_data[action_data['TECHNOLOGY_SWITCH']==0]['ACTION_ID'].tolist()
    else:
        return action_data['ACTION_ID'].tolist()
    
def get_router(customer_row, restrict_data, action_data):
        
    '''
    This function returns the posible 'router switch needed' actions
    
    params:
    Input: 
    
    customer_row: takes a single row from customer_data data frame of the column 'SPEED_MB', 'TECH', 'BEST_TECH' 
                  and 'ROUTER'
    restrict_data: tech restriction dataframe
    action_data: action dataframe
    
    Output:
    
    returns all possible actions(list) for promotion to do a router switch or not
    '''
    
    current_speed, current_tech, best_tech, current_router = customer_row
    
    x = restrict_data[(restrict_data['TECHNOLOGIE_GRUPPE'] == current_tech) & (restrict_data['BANDBREITE_MB']>current_speed)]
    if current_router in x['ENDGERAET'].unique():
        return action_data[action_data['ROUTER_SWITCH']==0]['ACTION_ID'].tolist()
    else:
        return action_data[action_data['ROUTER_SWITCH']==1]['ACTION_ID'].tolist()
    
def get_router_type(customer_row, restrict_data, action_data):
        
    '''
    This function returns the posible 'router needed' actions
    
    params:
    Input: 
    
    customer_row: takes a single row from customer_data data frame of the column 'SPEED_MB', 'TECH', 'BEST_TECH', 
                  'ROUTER' and 'ROUTER_TYPE'
    restrict_data: tech restriction dataframe
    action_data: action dataframe
    
    Output:
    
    returns all possible actions(list) for promotion based on router type
    '''
    
    
    current_speed, current_tech, best_tech, current_router, current_router_type = customer_row
    
    x = restrict_data[(restrict_data['TECHNOLOGIE_GRUPPE'] == current_tech) & (restrict_data['BANDBREITE_MB']>current_speed)]
    
    if current_router not in x['ENDGERAET'].unique():
        if current_router_type in ['basic', 'unknown']:
            return action_data[action_data['ROUTER_TYPE'].isin(['basic', 'premium'])]['ACTION_ID'].tolist()
        if current_router_type == 'premium':
            return action_data[action_data['ROUTER_TYPE'] == 'premium']['ACTION_ID'].tolist()
    else:
        return action_data[action_data['ROUTER_TYPE'] == 'no change']['ACTION_ID'].tolist()
        
        
    

In [7]:
all_actions = {k:[] for k in action_data['ACTION_ID'].values}

for i, row in customer_data.iloc[:,:9].iterrows():

   
    actions = []
    
    actions.extend(get_promotion(row['PRODUCT'], action_data))
    actions.extend(get_speed([row['SPEED_MB'], row['TECH']], restrict_data, action_data))
    actions.extend(get_technology([row['TECH'], row['BEST_TECH']], action_data))
    actions.extend(get_router([row['SPEED_MB'], row['TECH'], row['BEST_TECH'], row['ROUTER']], restrict_data, action_data))
    actions.extend(get_router_type([row['SPEED_MB'], row['TECH'], row['BEST_TECH'], row['ROUTER'], row['ROUTER_TYPE']], restrict_data, action_data))
    
    actions = sorted(list(set(actions)))
    
    all_no_keys = [k for k in all_actions.keys() if k not in actions]
    
    for act in actions:
        all_actions[act].append(True)

    for a in all_no_keys:
        all_actions[a].append(False)
        
        

#### Step 5:  Join the dataframes to give complete solution

In [8]:
complete_customer_data = pd.concat([customer_data.iloc[:,:9], pd.DataFrame(all_actions)], axis=1)

In [9]:
complete_customer_data

,CUSTOMER_ID,PRODUCT_RANGE,PRODUCT,CONTRACT_START_DT,SPEED_MB,TECH,BEST_TECH,ROUTER,ROUTER_TYPE,A_01,...,A_51,A_52,A_53,A_54,A_55,A_56,A_57,A_58,A_59,A_60
0,329 08 509,2015,Single Flat,10.11.2019,100,Cable,Cable,Arris TG2492S,basic,True,...,True,True,True,True,True,True,True,True,True,True
1,235 09 840,2018,Double Flat,08.03.2018,50,FTTC,FTTC,AVM FRITZ!Box 7430,basic,True,...,False,True,True,True,True,False,False,True,True,True
2,192 85 309,2015,Tripple Flat,21.07.2019,25,BSA,BSA,unknown,unknown,True,...,True,True,True,True,False,True,True,True,True,True
3,420 95 309,2012,Single Flat,22.09.2019,50,FTTH,FTTH,AVM FRITZ!Box 7390,basic,True,...,True,True,True,True,True,True,True,True,True,True
4,309 50 932,2018,Double Flat,15.01.2020,18,FTTC,FTTH,AVM FRITZ!Box 7590,premium,True,...,True,True,True,True,True,True,True,True,True,True
5,239 85 092,2018,Single Flat,18.12.2018,25,BSA,FTTH,AVM FRITZ!Box 7583,premium,True,...,True,True,True,True,True,True,True,True,True,True
6,230 95 081,2012,Double Flat,05.05.2019,100,FTTH,FTTH,AVM FRITZ!Box 7390,basic,True,...,True,True,True,True,True,True,True,True,True,True
7,595 80 238,2015,Double Flat,21.11.2019,100,FTTH,FTTH,unknown,unknown,True,...,True,True,True,True,True,True,True,True,True,True
8,239 50 098,2018,Double Flat,07.06.2019,50,FTTH,FTTH,unknown,unknown,True,...,True,True,True,True,True,True,True,True,True,True
9,915 80 908,2018,Single Flat,23.08.2019,25,BSA,G.fast,AVM FRITZ!Box 7530,basic,True,...,True,True,True,True,True,True,True,True,True,True


In [ ]:
complete_customer_data.to_csv('solution_customer_data.csv', index=False)